In [13]:
import zipfile as zp
import pandas as pd
# from pypac import PACSession as Session #or use requests below if non-ONS
#from requests import Session
from io import BytesIO
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import LoadData as dataLoader
%matplotlib notebook

###  Read in files


In [4]:
# Path to zip files
path = "/Users/lb8075/Loneliness_Data_challenge/Loneliness_notebook/Data/"

In [3]:
print (path[:-5])
print (path[:-25])

/Users/lb8075/Loneliness_Data_challenge/Loneliness_notebook/
/Users/lb8075/Loneliness_Data_challenge/


First lets load all the data... do this just once!

In [14]:
proc_data, proc_data_dict =dataLoader.loadFinalData('../Loneliness')
codes=dataLoader.loadProccessedData('../Loneliness')
drugs_list = dataLoader.loadDrugsList('../Loneliness')
msoa_data, msoa_data_dict = dataLoader.loadmsoaData('../Loneliness')

In [10]:
proc_data.head()

,pcstrip,Year,NUMBER_OF_PATIENTS,SHA,PCT,oseast1m,osnrth1m,lsoa11,msoa11,ru11ind,...,alzheimers_zscore,blood pressure_zscore,hypertension_zscore,diabetes_zscore,cardiovascular disease_zscore,insomnia_zscore,addiction_zscore,social anxiety_zscore,loneliness_zscore,loneills
0,AL100BS,2016,3171.916667,Q58,06K,522443,208996,E01023927,E02004990,C1,...,-0.611385,0.281695,-0.311210,-0.457039,-0.403666,-0.470144,-0.088685,-0.206785,-0.696324,-2.090846
1,AL100BS,2017,7990.750000,Q58,06K,522443,208996,E01023927,E02004990,C1,...,-0.092288,0.467674,-0.262544,-0.275512,-0.192223,-0.434156,-0.086022,0.199294,-0.315757,-0.894752
2,AL100BS,2018,9624.416667,Q58,06K,522443,208996,E01023927,E02004990,C1,...,0.196357,0.709112,-0.041739,0.129947,0.097690,-0.526029,-0.088685,0.937151,0.328936,0.616736
3,AL100NL,2016,4195.000000,Q58,06K,522442,208808,E01023920,E02004991,C1,...,-0.590078,-0.046907,0.456131,-0.055416,-0.127068,-0.120109,-0.087082,0.213252,-0.086705,-0.262717
4,AL100NL,2017,10554.666667,Q58,06K,522442,208808,E01023920,E02004991,C1,...,-0.234218,-0.103745,0.397698,0.041207,-0.093135,0.088218,-0.088685,0.361831,0.026190,0.541874


In [16]:
codes.head()

,Unnamed: 0,PracCode,Town,Postcode,blood pressure,ITEMS,loneliness,social anxiety,insomnia,alzheimers,...,Street,hypertension,cardiovascular disease,PCT,Date,addiction,Town2,CenterName,depression,diabetes
0,0,A81002,STOCKTON ON TEES,TS18 2AW,2936,36817,9675,815,41,90,...,FARRER STREET,2412,2595,00K,201810,0,CLEVELAND,QUEENS PARK MEDICAL CTR,2866,711
1,1,A81004,MIDDLESBROUGH,TS5 8SB,1030,19808,4628,396,148,41,...,ACKLAM,1286,1254,00M,201810,0,,TRIMDON AVENUE,1308,389
2,2,A81005,GUISBOROUGH,TS14 7DJ,1230,17000,4274,316,42,35,...,RECTORY LANE,1236,1329,00M,201810,0,,SPRINGWOOD SURGERY,987,279
3,3,A81006,STOCKTON-ON-TEES,TS18 2AT,2073,30509,7953,809,17,73,...,TENNANT STREET,1669,2270,00K,201810,0,CLEVELAND,TENNANT ST MEDICAL PRACT,2438,558
4,4,A81007,HARTLEPOOL,TS24 7PW,889,18189,4087,415,4,32,...,PARK ROAD,1023,990,00K,201810,0,CLEVELAND,ONE LIFE HARTLEPOOL,1536,274


In [31]:
codes = codes.drop(['Unnamed: 0'], axis=1)
codes.head()

,PracCode,Town,Postcode,blood pressure,ITEMS,loneliness,social anxiety,insomnia,alzheimers,pcstrip,...,Street,hypertension,cardiovascular disease,PCT,Date,addiction,Town2,CenterName,depression,diabetes
0,A81002,STOCKTON ON TEES,TS18 2AW,2936,36817,9675,815,41,90,TS182AW,...,FARRER STREET,2412,2595,00K,201810,0,CLEVELAND,QUEENS PARK MEDICAL CTR,2866,711
1,A81004,MIDDLESBROUGH,TS5 8SB,1030,19808,4628,396,148,41,TS58SB,...,ACKLAM,1286,1254,00M,201810,0,,TRIMDON AVENUE,1308,389
2,A81005,GUISBOROUGH,TS14 7DJ,1230,17000,4274,316,42,35,TS147DJ,...,RECTORY LANE,1236,1329,00M,201810,0,,SPRINGWOOD SURGERY,987,279
3,A81006,STOCKTON-ON-TEES,TS18 2AT,2073,30509,7953,809,17,73,TS182AT,...,TENNANT STREET,1669,2270,00K,201810,0,CLEVELAND,TENNANT ST MEDICAL PRACT,2438,558
4,A81007,HARTLEPOOL,TS24 7PW,889,18189,4087,415,4,32,TS247PW,...,PARK ROAD,1023,990,00K,201810,0,CLEVELAND,ONE LIFE HARTLEPOOL,1536,274


In [24]:
merged_data= proc_data.merge( codes , on=['PCT','pcstrip','SHA'])

In [25]:
merged_data.head()

,pcstrip,Year,NUMBER_OF_PATIENTS,SHA,PCT,oseast1m,osnrth1m,lsoa11,msoa11,ru11ind,...,alzheimers,Street,hypertension,cardiovascular disease,Date,addiction,Town2,CenterName,depression,diabetes
0,AL100BS,2016,3171.916667,Q58,06K,522443,208996,E01023927,E02004990,C1,...,41,9/11 WELLFIELD ROAD,1014,1008,201810,0,HERTFORDSHIRE,WRAFTON HOUSE SURGERY,998,353
1,AL100BS,2016,3171.916667,Q58,06K,522443,208996,E01023927,E02004990,C1,...,39,9/11 WELLFIELD ROAD,947,982,201811,0,HERTFORDSHIRE,WRAFTON HOUSE SURGERY,922,334
2,AL100BS,2016,3171.916667,Q58,06K,522443,208996,E01023927,E02004990,C1,...,41,9/11 WELLFIELD ROAD,969,924,201812,0,HERTFORDSHIRE,WRAFTON HOUSE SURGERY,876,325
3,AL100BS,2017,7990.750000,Q58,06K,522443,208996,E01023927,E02004990,C1,...,41,9/11 WELLFIELD ROAD,1014,1008,201810,0,HERTFORDSHIRE,WRAFTON HOUSE SURGERY,998,353
4,AL100BS,2017,7990.750000,Q58,06K,522443,208996,E01023927,E02004990,C1,...,39,9/11 WELLFIELD ROAD,947,982,201811,0,HERTFORDSHIRE,WRAFTON HOUSE SURGERY,922,334


In [ ]:
proc_data.rename(index=str, columns={"pcstrip": "Postcode"})

In [ ]:
proc_data = proc_data.merge(codes, on='Postcode') 

In [ ]:
# proc_data=proc_data.loc[:,['oseast1m','osnrth1m','msoa11','insomnia_zscore','alzheimers_zscore','addiction_zscore','depression_zscore']]
# proc_data=proc_data.iloc[1:10000]

In [ ]:
insom=proc_data['insomnia_zscore']


In [ ]:
# proc_data['insomnia_zscore'].hist(bins=100,range=(-2,15))
# print (proc_data['insomnia_zscore'].mean())

In [ ]:
proc_data.dtypes

In [1]:
geo_data = gpd.GeoDataFrame(proc_data, 
                            crs = {'init':'epsg:27700'}, 
                            geometry = proc_data.apply(lambda geom: Point(geom['oseast1m'],geom['osnrth1m']),axis=1))
geo_data.head(100)

NameError: name 'gpd' is not defined

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (16,6), sharex = True, sharey = True)

geo_data[geo_data['Year'] == 2016].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax1);
geo_data[geo_data['Year'] == 2017].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax2);
geo_data[geo_data['Year'] == 2018].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax3);
